# ALIGNED project: GSA correlation tutorial

**Aligning Life Cycle Assessment methods and bio-based sectors for improved environmental performance**

[http://www.alignedproject.eu/](http://www.alignedproject.eu/)

_Horizon Europe grant agreement N° 101059430. Views and opinions expressed are however those of the author(s) only and do not necessarily reflect those of the European Union or the European Research Executive Agency._ 


## WP1 Shared modelling framework and learnings
### Task 1.4 Framework for interpreting uncertainty

#### Deliverable 1.2 Description of scientific methods

#### Tutorial for performing Global Sensitivity Analysis using the correlation approach

#### Massimo Pizzol, Aalborg University (AAU), 2024

This notebook show how to perform a simple Global Sensitivity Analysis (GSA) for a example product system of a biobased product and using correlation analysis to quantify the defree of sensitivity.

In [1]:
# Importing packages
import bw2data as bd
import bw2calc as bc
import pandas as pd
import numpy as np
from scipy import stats
from lci_to_bw2 import * # import all the functions of this module

In [2]:
# open a project with ecoinvent v.3.11 consequential system model
bd.projects.set_current('advlca25')

We start by importing data about a fictional ("dummy") product system for a biobased product.

In [3]:
# Import the dummy product system

# import data from csv

#mydata = pd.read_csv('LCI1-bw-format.csv', header = 0, sep = ",") # using csv file avoids encoding problem
mydata = pd.read_csv('ALIGNED-LCI-biobased-product-dummy.csv', header = 0, sep = ",") # using csv file avoids encoding problem
mydata.head()

# keep only the columns not needed
mydb = mydata[['Activity database','Activity code','Activity name','Activity unit','Activity type',
               'Exchange database','Exchange input','Exchange amount','Exchange unit','Exchange type',
               'Exchange uncertainty type','Exchange loc','Exchange scale','Exchange negative', 
               'Simapro name',	'Simapro unit', 'Simapro type']].copy()

mydb = mydata.copy()

mydb['Exchange uncertainty type'] = mydb['Exchange uncertainty type'].fillna(0).astype(int) # uncertainty as integers
# Note: to avoid having both nan and values in the uncertainty column I use zero as default

#print(mydb.head())

In [4]:
# Create dictionary in bw format and write database to disk. 
# Shut down all other notebooks using the same project before doing this
bw2_db = lci_to_bw2(mydb) # a function from the lci_to_bw2 module

# write database
bd.Database('ALIGNED-biob-prod-dummy').write(bw2_db)

11:08:36 [warning  ] Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 5/5 [00:00<00:00, 1545.09it/s]

11:08:36 [info     ] Vacuuming database            


The product system includes different activities such as the production, use, and end of life of the biobased product.

In [5]:
# check what foreground activities are included
for act in bd.Database('ALIGNED-biob-prod-dummy'):
    print(act, act['code'])

'Biobased-product-use' (year, None, None) f9eabf64-b899-40c0-9f9f-2009dbb0a0b2
'Biobased-product-manufacturing' (kilogram, None, None) a37d149a-6508-4563-8af6-e5a39b4176df
'Biobased-product-eol' (kilogram, None, None) c8301e73-d521-4a89-998b-30b7e7751011
'Biomass-growth' (kilogram, None, None) a7d34649-9c10-4423-bac3-ecab9b43b20c
'Biomass-processing' (kilogram, None, None) 403a5c32-c769-46fc-8b9a-74b8eb3c79d1


In [6]:
# More info 
myact = bd.Database('ALIGNED-biob-prod-dummy').get('f9eabf64-b899-40c0-9f9f-2009dbb0a0b2') # Biobased-product-use
myact._data

{'name': 'Biobased-product-use',
 'unit': 'year',
 'type': 'processwithreferenceproduct',
 'database': 'ALIGNED-biob-prod-dummy',
 'code': 'f9eabf64-b899-40c0-9f9f-2009dbb0a0b2',
 'id': 161415763142123521}

In [7]:
# Uncertainty is also there
list(myact.exchanges())[1]._data

{'input': ('ALIGNED-biob-prod-dummy', 'a37d149a-6508-4563-8af6-e5a39b4176df'),
 'amount': 50.0,
 'unit': 'kilogram',
 'type': 'technosphere',
 'uncertainty type': 4,
 'minimum': 37.5,
 'maximum': 62.5,
 'Notes': 'Input of manufacturing',
 'Simapro name': 'Biobased-product-manufacturing',
 'Simapro unit': 'kg',
 'output': ('ALIGNED-biob-prod-dummy', 'f9eabf64-b899-40c0-9f9f-2009dbb0a0b2')}

We calculate a static climate impact score for the fictional biobased product, to be used for reference later on.

In [8]:
# calculation of static LCA score
mymethod = ('ecoinvent-3.11', 'IPCC 2021', 'climate change: fossil', 'global warming potential (GWP100)')
myact = bd.Database('ALIGNED-biob-prod-dummy').get('f9eabf64-b899-40c0-9f9f-2009dbb0a0b2') # Biobased-product-use
functional_unit = {myact: 1}
LCA = bc.LCA(functional_unit, mymethod)
LCA.lci()
LCA.lcia()
print(LCA.score)

180.28497357367456


### Now perform global sensitivity analysis

The procedure is in three steps.

1) A set of model input parameters is chosen. These are **values of specific exchanges**. A sample of values is produced for each model input, in this case, only 5 values.
2) A simulation is performed. Initial prameter values are substituted with those in the sample, iteratively, and new model outputs, that are LCA scores, are calculated.
3) A correlation is estimated between model input values and output values.

#### Step 1
A sample of values for each parameter.

In [9]:
# Can be done in many ways, here very basic using lists.
par1_values = [-1.25, -1.10, -1, -0.9, -0.75] # In bomass growth, value of CO2 uptake
par2_values = [0.1, 0.3, 0.5, 0.7, 0.9] # In biomass processing, amount of energy used.
par3_values = [20,30,40,50,60]  # In use of product, amount of manufactured product needed
par4_values = [1.25, 1.10, 1, 0.9, 0.75] # In use of product, amount of manufactured product needed

Associate the values to specific exchanges using the coordinates (column and row) of the technosphere (A) and biosphere (B) matrices

In [10]:
param_samples = [(('ALIGNED-biob-prod-dummy', 'a7d34649-9c10-4423-bac3-ecab9b43b20c'), 
  ('ecoinvent-3.11-biosphere', '349b29d1-3e58-4c66-98b9-9d1a076efd2e'), par1_values), # In bomass growth, value of CO2 uptake
(('ALIGNED-biob-prod-dummy', '403a5c32-c769-46fc-8b9a-74b8eb3c79d1'),
 ('ecoinvent-3.11-consequential', 'a69490974731f9da34043d5c926cd167'), par2_values), # In biomass processing, amount of energy used.
 (('ALIGNED-biob-prod-dummy', 'f9eabf64-b899-40c0-9f9f-2009dbb0a0b2'),
  ('ALIGNED-biob-prod-dummy', 'a37d149a-6508-4563-8af6-e5a39b4176df'), par3_values), # In use of product, amount of manufactured product needed
  (('ALIGNED-biob-prod-dummy', 'c8301e73-d521-4a89-998b-30b7e7751011'),
   ('ecoinvent-3.11-biosphere', '349b29d1-3e58-4c66-98b9-9d1a076efd2e'), par4_values)] # In end of life, amount of CO2 released

#### (technical note)

We can do the same in a more elegant way, taking the data directly from the BW database that we just created...


In [11]:
# In biomass growth, value of CO2 uptake
par1 = list(bd.Database('ALIGNED-biob-prod-dummy').get('a7d34649-9c10-4423-bac3-ecab9b43b20c').exchanges())[3]
# In biomass processing, amount of energy used.
par2 = list(bd.Database('ALIGNED-biob-prod-dummy').get('403a5c32-c769-46fc-8b9a-74b8eb3c79d1').exchanges())[1]
# In use of product, amount of manufactured product needed
par3 = list(bd.Database('ALIGNED-biob-prod-dummy').get('f9eabf64-b899-40c0-9f9f-2009dbb0a0b2').exchanges())[1]
# In use of product, amount of manufactured product needed
par4 = list(bd.Database('ALIGNED-biob-prod-dummy').get('c8301e73-d521-4a89-998b-30b7e7751011').exchanges())[2]

n_iter = 5
param_samples = [(par1['output'], par1['input'], par1.random_sample(n = n_iter)),
                 (par1['output'], par2['input'], par2.random_sample(n = n_iter)),
                 (par1['output'], par3['input'], par3.random_sample(n = n_iter)),
                 (par1['output'], par4['input'], par4.random_sample(n = n_iter))]

Let's look at the samples obtained

In [12]:
param_samples

[(('ALIGNED-biob-prod-dummy', 'a7d34649-9c10-4423-bac3-ecab9b43b20c'),
  ('ecoinvent-3.11-biosphere', '349b29d1-3e58-4c66-98b9-9d1a076efd2e'),
  array([-0.76595462, -1.06695893, -0.88963154, -1.20323493, -1.04126252])),
 (('ALIGNED-biob-prod-dummy', 'a7d34649-9c10-4423-bac3-ecab9b43b20c'),
  ('ecoinvent-3.11-consequential', 'a69490974731f9da34043d5c926cd167'),
  array([0.40004861, 0.45875467, 0.49982007, 0.50283803, 0.43173111])),
 (('ALIGNED-biob-prod-dummy', 'a7d34649-9c10-4423-bac3-ecab9b43b20c'),
  ('ALIGNED-biob-prod-dummy', 'a37d149a-6508-4563-8af6-e5a39b4176df'),
  array([62.13814778, 45.52723927, 45.45383527, 48.05859881, 50.3156843 ])),
 (('ALIGNED-biob-prod-dummy', 'a7d34649-9c10-4423-bac3-ecab9b43b20c'),
  ('ecoinvent-3.11-biosphere', '349b29d1-3e58-4c66-98b9-9d1a076efd2e'),
  array([1.22704136, 1.03768889, 1.01804711, 1.1416563 , 1.12582441]))]

#### Step 2

Iteration through all parameter samples. Input values are replaced and new impact scores are calculated at each iteration.

In [41]:
# testing the loop that will be used for the simulation, iterate through parameter values
    
for i in range(0,n_iter): # iterate 5 times...
    for s in param_samples: # and for each paramater...
        
        # Get the 'id' using the activity object
        col_id = bd.Database(s[0][0]).get(s[0][1]).id
        row_id = bd.Database(s[1][0]).get(s[1][1]).id

        
        # Use the id and the mapping dictionaries to find matrix row and columns
        if s[1][0] == 'ecoinvent-3.11-biosphere':
            col = LCA.dicts.activity[col_id] # find column index of A matrix for the activity
            row = LCA.dicts.biosphere[row_id] # find row index of B matrix for the exchange
            print(f"biosphere value: {LCA.biosphere_matrix[row,col]} | paramater value: {s[2][i]}")
        else:
            col = LCA.dicts.activity[col_id] # find column index of A matrix for the activity
            row = LCA.dicts.activity[row_id] # find row index of A matrix for the exchange
            print(f"technosphere value: {LCA.technosphere_matrix[row,col]} | paramater value: {-s[2][i]}") # CHANGE OF SIGN!
        
    print("* * *")

biosphere value: -1.0 | paramater value: -0.7659546182429033
technosphere value: 0.0 | paramater value: -0.4000486120989718
technosphere value: 0.0 | paramater value: -62.13814777860145
biosphere value: -1.0 | paramater value: 1.227041364823212
* * *
biosphere value: -1.0 | paramater value: -1.0669589270744357
technosphere value: 0.0 | paramater value: -0.4587546698769527
technosphere value: 0.0 | paramater value: -45.527239267669266
biosphere value: -1.0 | paramater value: 1.0376888884217577
* * *
biosphere value: -1.0 | paramater value: -0.8896315389985598
technosphere value: 0.0 | paramater value: -0.49982007289101615
technosphere value: 0.0 | paramater value: -45.45383527455697
biosphere value: -1.0 | paramater value: 1.018047113624029
* * *
biosphere value: -1.0 | paramater value: -1.2032349307070134
technosphere value: 0.0 | paramater value: -0.5028380334752284
technosphere value: 0.0 | paramater value: -48.05859880894107
biosphere value: -1.0 | paramater value: 1.141656297126976

In [14]:
# Implementing the loop
import copy # a library helps to save the original matrices

GSA_value_results = []

for i in range(0,n_iter): 
    LCA.lci()
    LCA.lcia()
    print("initial value", LCA.score)
    
    old_bio_matrix = copy.deepcopy(LCA.biosphere_matrix)
    old_tech_matrix = copy.deepcopy(LCA.technosphere_matrix)
    
    for s in param_samples:
        col_id = bd.Database(s[0][0]).get(s[0][1]).id
        row_id = bd.Database(s[1][0]).get(s[1][1]).id

        if s[1][0] == 'ecoinvent-3.11-biosphere':
            col = LCA.dicts.activity[col_id] # find column index of A matrix for the activity
            row = LCA.dicts.biosphere[row_id] # find row index of B matrix for the exchange
            new_bio = s[2][i]
            LCA.biosphere_matrix[row,col] = new_bio # substitute the value
        else:
            col = LCA.dicts.activity[col_id] # find column index of A matrix for the activity
            row = LCA.dicts.activity[row_id] # find row index of A matrix for the exchange
            new_tech = -s[2][i]
            LCA.technosphere_matrix[row,col] = new_tech # substitute the value
                
    LCA.redo_lci() # uses the new A matrix
    LCA.lcia()
    GSA_value_results.append(LCA.score)
    print('end value', LCA.score)
    print("---")

    # Restore the original matrices
    LCA.biosphere_matrix = old_bio_matrix
    LCA.technosphere_matrix = old_tech_matrix



initial value 180.28497357367456


/opt/miniconda3/envs/bw25/lib/python3.13/site-packages/scipy/sparse/_index.py:155: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil and dok are more efficient.
  self._set_intXint(row, col, x.flat[0])


end value 44.79417146262437
---
initial value 180.28497357367456
end value 42.65635967873796
---
initial value 180.28497357367456
end value 42.66314111474569
---
initial value 180.28497357367456
end value 42.998007601962414
---
initial value 180.28497357367456
end value 43.38910174433519
---


#### Step 3

Now we have both the input and output values and we can look at their correlation.

To estimate the degree of correlation use a Pearson coefficient of linear relationship between two sets of values.

See here for info: 
https://en.wikipedia.org/wiki/Pearson_correlation_coefficient
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html

This approach has for example been used in:

_Kim, A., Mutel, C., Froemelt, A., 2021. Robust high-dimensional screening. Environmental Modelling & Software 105270._ https://doi.org/10.1016/j.envsoft.2021.105270


In [15]:
# calcualte correlation

corr_data = pd.DataFrame([i[2] for i in param_samples], index = ['par1','par2', 'par3', 'par4']).T
corr_data['GWI'] = GSA_value_results
corr_data.corr()

,par1,par2,par3,par4,GWI
par1,1.000000,-0.570894,0.643742,0.262231,0.633671
par2,-0.570894,1.000000,-0.812483,-0.647388,-0.826205
par3,0.643742,-0.812483,1.000000,0.899324,0.998476
par4,0.262231,-0.647388,0.899324,1.000000,0.904875
GWI,0.633671,-0.826205,0.998476,0.904875,1.000000


In this specific case "par3"  is the parameter to which the results are most sensitive to.


In [16]:
# what was "par3" again?
par3 # amount of manufactured product need din the use stage

Exchange: 50.0 kilogram 'Biobased-product-manufacturing' (kilogram, None, None) to 'Biobased-product-use' (year, None, None)>